## 2) Implement a “grid search”

Investigate the follow parameters:

- activation units (sigmoid, relu, elu, etc.),

- different minimization algorithms (ADAM, RMSprop, Nesterov, etc.),

- different dropouts

- batch normalization

- different size or number of layers

In [ ]:
import tensorflow        as tf
import numpy             as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 1
%aimport nn_model
%aimport create_data
%aimport preprocess

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
tf.__version__

'2.8.0'

In [ ]:
# number of data
N = 5000
# box size
B = 100

# random seed for reproducibility
np.random.seed(20220308)

x = create_data.sample_data(N, B)
y = create_data.assign_label(x)

mean = np.mean(x, axis=0)
std = np.std(x, axis=0)

# re-scaling data 
x_scaled = preprocess.standardize(x, mean, std)

In [ ]:
train_frac = 0.7

N_train = int(N*train_frac)

training_data, training_labels = np.copy(x_scaled[0:N_train]), np.copy(y[0:N_train])
test_data,     test_labels     = np.copy(x_scaled[N_train:]),  np.copy(y[N_train:])

print(f"Traing set :\t{training_data.shape[0]}\nTest set  :\t{test_data.shape[0]}")

N_valid = 500

x_train, y_train = np.copy(training_data[N_valid:]),  np.copy(training_labels[N_valid:])
x_valid, y_valid = np.copy(training_data[0:N_valid]), np.copy(training_labels[0:N_valid])

print(f"\nN_train :\t{x_train.shape[0]} \nN_valid :\t{x_valid.shape[0]}")

Traing set :	3500
Test set  :	1500

N_train :	3000 
N_valid :	500


In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(
    n_features,
    neurons           = [20, 20, 1],
    dropout_arc       = [1],
    dropout_rates     = [0.2],
    batch_norm_arc    = [2],
    hidden_activation = "relu",
    output_activation = "sigmoid",
    name              = "my lovely neural network",
    loss              = "binary_crossentropy",
    optimizer         = "adam",
    metrics           = ["accuracy"]
):
    # initialize architecture
    architecture = [n_features] + neurons
    input_dim    = n_features

    # create model
    network = nn_model.NN(
        input_dim         = input_dim,
        architecture      = architecture,
        dropout_layers    = dropout_arc,
        dropout_rates     = dropout_rates,
        batch_norm_layers = batch_norm_arc,
        hidden_activation = hidden_activation,
        output_activation = output_activation,
        nn_name           = name
    ) 

    # solves the "Model cannot be saved" issue
    network.compute_output_shape((None, input_dim))   
    
    network.compile(
        loss      = loss,
        optimizer = optimizer,
        metrics   = metrics,
    )
    return network

In [ ]:
minibatch = 100
N_epochs  = 300

model_gridsearch = KerasClassifier(build_fn=create_model, n_features=x.shape[0], epochs=N_epochs, batch_size=minibatch, 
                                   validation_data = (x_valid, y_valid))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """


In [ ]:
# list of allowed optional arguments for the optimizer, see `compile_model()`
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# define parameter dictionary
param_grid = dict(optimizer=optimizer)
param_grid = {"epochs": [100, 200]}
# call scikit grid search module
grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_train,y_train)

Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/100
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
8 fits failed out of a total of 8.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/wrappers/scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/sh

ValueError: in user code:

    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "nn_12" (type NN).
    
    in user code:
    
        File "

#### tests trying to hardcode a gridsearch, gave up because of:
- should implement also cv
- seems that the built in function is ok with our requests

In [ ]:
n_features = x.shape[1]

# NN hyperparameters
architecture      = [n_features, 20, 20, 1]
dropout_arc       = [1]
dropout_rates     = [0.2]
batch_norm_arc    = [2]   
input_dim         = architecture[0]
hidden_activation = "relu"
output_activation = "sigmoid"
name              = "my lovely neural network"

# build the NN model
network = nn_model.NN(
    input_dim         = input_dim,
    architecture      = architecture,
    dropout_layers    = dropout_arc,
    dropout_rates     = dropout_rates,
    batch_norm_layers = batch_norm_arc,
    hidden_activation = hidden_activation,
    output_activation = output_activation,
    nn_name           = name,
)

# print summary
network.summary()

# compile NN with loss, optimizer and metrics
loss      = "binary_crossentropy"
optimizer = "adam"
metrics   = ["accuracy"]

network.compile(
    loss      = loss,
    optimizer = optimizer,
    metrics   = metrics,
)

In [ ]:
activation_units = np.array(["sigmoid", "relu"])
layers           = np.array([1, 2, 3, 4])
optimizer        = np.array(["adam", "nestorov", "rms_prop"])
X, Y, Z = np.meshgrid(activation_units, layers, optimizer)

In [ ]:
grid_params = {}

grid_params["activation_units"] = np.array(["sigmoid", "relu"])
grid_params["optimizer"]        = np.array(["adam", "nestorov", "rms_prop"])
grid_params["layers"]           = [1,2]

In [ ]:
from itertools import product
for params in product(*grid_params.values()):
    print(dict(zip(grid_params.keys(), params)))

{'activation_units': 'sigmoid', 'optimizer': 'adam', 'layers': 1}
{'activation_units': 'sigmoid', 'optimizer': 'adam', 'layers': 2}
{'activation_units': 'sigmoid', 'optimizer': 'nestorov', 'layers': 1}
{'activation_units': 'sigmoid', 'optimizer': 'nestorov', 'layers': 2}
{'activation_units': 'sigmoid', 'optimizer': 'rms_prop', 'layers': 1}
{'activation_units': 'sigmoid', 'optimizer': 'rms_prop', 'layers': 2}
{'activation_units': 'relu', 'optimizer': 'adam', 'layers': 1}
{'activation_units': 'relu', 'optimizer': 'adam', 'layers': 2}
{'activation_units': 'relu', 'optimizer': 'nestorov', 'layers': 1}
{'activation_units': 'relu', 'optimizer': 'nestorov', 'layers': 2}
{'activation_units': 'relu', 'optimizer': 'rms_prop', 'layers': 1}
{'activation_units': 'relu', 'optimizer': 'rms_prop', 'layers': 2}


In [ ]:
for param_label, param_value in grid_params.items():
    print(param_value)

['sigmoid' 'relu']
['adam' 'nestorov' 'rms_prop']


In [ ]:
l = np.meshgrid(*grid_params.values())
l[1].shape

(3, 2, 2)

In [ ]:
class GridSearch():

    def __init__(
        self, 
        grid_params, 
        fixed_params, 
        nn_params = ["activation_units", "optimizer", "loss"]
        ):
        """
        Grid Search

        Arguments:
        grid_params         [dict]    -> dict of parameters to evaluate        
        fixed_params        [dict]    -> dict of fixed parameters of the NN model
        nn_params           [list]    -> labels of total number of parameters for the NN model
        """
        
        self.grid_params  = grid_params
        self.fixed_params = fixed_params
        self.nn_params    = nn_params
        
        # build grid
        self.grid         = np.meshgrid(*self.grid_params.values())
        
    def fit(
        x_train,
        y_train,
        epochs,
        batch_size,
        validation_data,
        verbose
        ):
        """
        perform the grid search
        """
        
        # save fixed parameters
        for param_label, param_value in fixed_params.items(): 
            globals()[param_label] = param_value
        
        # loop over the grid -> come minchia fargli fare n nested loops con n variabile?! 
        #                       ( idea, funzione esterna (ricorsiva?))
        """
        idea: go through the whole grid and perfrom a NN model fit at each point
        """
        for


"""
additionals: 2d plot of grid search performance given 2 params with spheres

"""

SyntaxError: invalid syntax (<ipython-input-34-476ff68ca75c>, line 46)

In [ ]:
grid = GridSearch(grid_params)

activation_units
optimizer
['sigmoid' 'relu']


In [ ]:
# NN fixed hyperparameters 
architecture      = [n_features, 20, 20, 1]
dropout_arc       = [1]
dropout_rates     = [0.2]
batch_norm_arc    = [2]   
input_dim         = architecture[0]
hidden_activation = "relu"
output_activation = "sigmoid"
name              = "my lovely neural network"

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=452e6836-b7b8-455e-80d9-d4109b6baddf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>